In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import interpolate

In [42]:
df_actual=pd.read_csv('../Data/Actuals.csv')

df_actual = df_actual.drop(columns=['TICKER'])

df_actual = df_actual[df_actual['PENDS'] > '2014-12-31']

In [43]:
appl = df_actual[df_actual["OFTIC"] == "AAPL"]
appl.isna().sum()

OFTIC        0
CNAME        0
PENDS        0
MEASURE      0
PDICITY      0
ANNDATS      0
VALUE      131
dtype: int64

In [44]:
#min period end date'i 2015 sonrası olan yani 2015'ten sonra başlayan
df_actual['PENDS'] = pd.to_datetime(df_actual['PENDS']) #.year kullanabilmek için datetime çevir

min_pend = df_actual.groupby('OFTIC')['PENDS'].min()

# Filter companies with a minimum announce date after 2015
companies_to_keep = min_pend[min_pend.dt.year <= 2015].reset_index()['OFTIC']

# Create a new DataFrame with only the data for the selected companies
df_actual = df_actual[df_actual['OFTIC'].isin(companies_to_keep)]

In [45]:
df_actual.reset_index(drop=True,inplace=True)
df_actual

,OFTIC,CNAME,PENDS,MEASURE,PDICITY,ANNDATS,VALUE
0,PAYC,PAYCOM SOFTWARE,2015-03-31,EPS,QTR,2015-05-06,0.120
1,PAYC,PAYCOM SOFTWARE,2015-06-30,EPS,QTR,2015-08-04,0.100
2,PAYC,PAYCOM SOFTWARE,2015-09-30,EPS,QTR,2015-11-03,0.080
3,PAYC,PAYCOM SOFTWARE,2015-12-31,EPS,QTR,2016-02-09,0.100
4,PAYC,PAYCOM SOFTWARE,2016-03-31,EPS,QTR,2016-05-03,0.330
...,...,...,...,...,...,...,...
319468,TJX,TJX,2022-04-30,SAL,QTR,2022-05-18,11406.474
319469,TJX,TJX,2022-07-31,SAL,QTR,2022-08-17,11843.008
319470,TJX,TJX,2022-10-31,SAL,QTR,2022-11-16,12166.286
319471,TJX,TJX,2023-01-31,SAL,QTR,2023-02-22,14520.000


In [46]:
df_actual["MEASURE"].unique()

array(['EPS', 'BPS', 'CPS', 'CPX', 'CSH', 'DPS', 'EBG', 'EBI', 'EBS',
       'EBT', 'ENT', 'FFO', 'GPS', 'GRM', 'NAV', 'NDT', 'NET', 'OPR',
       'PRE', 'ROA', 'ROE', 'SAL'], dtype=object)

In [47]:
comp_list = df_actual["OFTIC"].unique()
comp_list

array(['PAYC', 'ANET', 'KEYS', 'MTCH', 'AAPL', 'AAP', 'ABBV', 'ABT',
       'ADSK', 'TAP', 'CB', 'ACN', 'ADBE', 'ADI', 'ADM', 'AEP', 'AES',
       'AFL', 'SWKS', 'HES', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'HON',
       'ALLE', 'ALGN', 'ALK', 'ALL', 'AMAT', 'PLD', 'AMD', 'AMGN', 'AMP',
       'AAL', 'AMT', 'AMZN', 'ANSS', 'AVGO', 'APA', 'APD', 'APH', 'ARE',
       'ABC', 'A', 'ADP', 'AVB', 'ATVI', 'AVY', 'AWK', 'AXP', 'AZO',
       'PNW', 'BA', 'BAX', 'BR', 'BBY', 'RHI', 'BDX', 'VZ', 'BF.B', 'BIO',
       'BK', 'BRK.B', 'WRB', 'BLK', 'BMY', 'BSX', 'BWA', 'BXP', 'CAG',
       'CAT', 'CBOE', 'C', 'CCL', 'CHTR', 'COST', 'STZ', 'CAH', 'CDW',
       'CE', 'FIS', 'CF', 'CFG', 'JPM', 'CHRW', 'CVX', 'CI', 'CINF', 'CL',
       'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMS', 'CNC', 'COF', 'CPB',
       'CPRT', 'CPT', 'CHD', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT',
       'CTSH', 'CMI', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DGX', 'TGT', 'DIS',
       'DISH', 'DLR', 'DLTR', 'DHR', 'DPZ', 'DG', 'DOV', 'DO

In [48]:
company_drop = ['AAL', 'AAP', 'ABBV', 'ABC', 'ABT', 'ADP', 'AFL', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'AMD', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVY', 'BA', 'BAX', 'BDX', 'BLK', 'BMY', 'BR', 'BRO', 'BSX', 'BWA', 'BXP', 'CAH', 'CAT', 'CB', 'CBOE', 'CCI', 'CDNS', 'CDW', 'CE', 'CF', 'CHD', 'CHRW', 'CHTR', 'CI', 'CL', 'CLX', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COP', 'CPT', 'CRL', 'CSX', 'CTLT', 'CTSH', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DGX', 'DHI', 'DHR', 'DIS', 'DISH', 'DLR', 'DOV', 'DOW', 'DPZ', 'DVA', 'DVN', 'DXCM', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EL', 'EMN', 'EMR', 'ENPH', 'EOG', 'EPAM', 'EQIX', 'EQR', 'ES', 'ESS', 'ETN', 'ETSY', 'EW', 'EXC', 'EXPD', 'EXPE', 'EXR', 'F', 'FANG', 'FAST', 'FCX', 'FE', 'FFIV', 'FIS', 'FISV', 'FLT', 'FMC', 'FOXA', 'FRT', 'FSLR', 'FTNT', 'GD', 'GE', 'GLW', 'GM', 'GNRC', 'GOOG', 'GOOGL', 'GPC', 'GPN', 'GRMN', 'GS', 'GWW', 'HAL', 'HAS', 'HES', 'HII', 'HLT', 'HOLX', 'HON', 'HSIC', 'HST', 'HSY', 'HUM', 'IBM', 'ICE', 'IEX', 'IFF', 'ILMN', 'INTC', 'IP', 'IPG', 'IR', 'IRM', 'ISRG', 'ITW', 'IVZ', 'JBHT', 'JCI', 'JKHY', 'JNJ', 'JNPR', 'K', 'KHC', 'KIM', 'KLAC', 'KMB', 'KMI', 'KO', 'LDOS', 'LH', 'LLY', 'LMT', 'LRCX', 'LUV', 'LVS', 'LYB', 'LYV', 'MA', 'MAA', 'MAR', 'MAS', 'MCD', 'MCHP', 'MCK', 'MCO', 'MDLZ', 'MET', 'MHK', 'MKTX', 'MLM', 'MMC', 'MMM', 'MNST', 'MO', 'MOH', 'MOS', 'MPC', 'MPWR', 'MRK', 'MRO', 'MS', 'MSCI', 'MSFT', 'MSI', 'MTCH', 'MTD', 'NCLH', 'NDAQ', 'NEE', 'NEM', 'NFLX', 'NI', 'NOC', 'NOW', 'NSC', 'NUE', 'NVR', 'NWL', 'NWSA', 'NXPI', 'O', 'ODFL', 'OKE', 'OMC', 'ON', 'ORLY', 'OXY', 'PAYC', 'PCAR', 'PEP', 'PFE', 'PFG', 'PG', 'PH', 'PHM', 'PKG', 'PKI', 'PLD', 'PNR', 'PPG', 'PRU', 'PSA', 'PSX', 'PWR', 'PXD', 'PYPL', 'QCOM', 'QRVO', 'RCL', 'RE', 'REG', 'RHI', 'RJF', 'RL', 'RMD', 'ROK', 'ROL', 'ROP', 'RSG', 'SBAC', 'SBUX', 'SCHW', 'SEDG', 'SEE', 'SHW', 'SLB', 'SO', 'SPG', 'STE', 'STLD', 'STX', 'SWK', 'SWKS', 'SYK', 'SYY', 'T', 'TAP', 'TDG', 'TECH', 'TEL', 'TER', 'TFX', 'TMO', 'TMUS', 'TRGP', 'TROW', 'TRV', 'TSCO', 'TSLA', 'TSN', 'TTWO', 'TXN', 'TXT', 'UAL', 'UDR', 'UHS', 'UNH', 'UNP', 'UPS', 'URI', 'V', 'VFC', 'VLO', 'VMC', 'VRSK', 'VRTX', 'VTR', 'VZ', 'WAT', 'WDC', 'WHR', 'WM', 'WMB', 'WRK', 'WST', 'WTW', 'WY', 'XOM', 'XRAY', 'YUM', 'ZBH', 'ZBRA', 'ZION','AEP', 'ALL', 'ANSS', 'AWK', 'PNW', 'CMA', 'MTB', 'HCA', 'KMX', 'LKQ', 'MGM', 'XEL', 'ACGL', 'TRMB', 'VRSN', 'VRSN', 'WAB', 'ADM', 'AES', 'AIG', 'ALLE', 'AXP', 'BIO', 'BK', 'BRK.B', 'BRK.B', 'WRB', 'C', 'CFG', 'JPM', 'CINF', 'COF', 'CPRT', 'DFS', 'DTE', 'DUK', 'EQT', 'RF', 'FITB', 'USB', 'FRC', 'IT', 'GILD', 'HBAN', 'HIG', 'HRL', 'BIIB', 'IDXX', 'IDXX', 'INCY', 'LNC', 'L', 'AME', 'ETR', 'BAC', 'WFC', 'NRG', 'NTRS', 'FOX', 'FOX', 'FOX', 'PCG', 'PEG', 'PTC', 'PM', 'PNC', 'POOL', 'PPL', 'PGR', 'REGN', 'CSGP', 'SBNY', 'EIX', 'SRE', 'SIVB', 'GEN', 'SNA', 'KEY', 'STT', 'SYF', 'TDY', 'TYL', 'TYL', 'AEE', 'WEC', 'LNT', 'WYNN', 'XYL', 'ZTS']
measures = ["EPS","ROE","SAL","NET","EBI","EBT","GPS","DPS","BPS","NAV"]

df_actual = df_actual[df_actual["MEASURE"].isin(measures)]
print(df_actual["MEASURE"].unique())
print(len(company_drop))
print(len(df_actual["OFTIC"].unique()))
df_actual = df_actual[~df_actual["OFTIC"].isin(company_drop)]
df_actual

['EPS' 'BPS' 'DPS' 'EBI' 'EBT' 'GPS' 'NAV' 'NET' 'ROE' 'SAL']
413
459


,OFTIC,CNAME,PENDS,MEASURE,PDICITY,ANNDATS,VALUE
66,KEYS,KEYSIGHT TECH,2015-01-31,EPS,QTR,2015-02-19,0.560
67,KEYS,KEYSIGHT TECH,2015-04-30,EPS,QTR,2015-05-19,0.700
68,KEYS,KEYSIGHT TECH,2015-07-31,EPS,QTR,2015-08-19,0.550
69,KEYS,KEYSIGHT TECH,2015-10-31,EPS,QTR,2015-11-19,0.710
70,KEYS,KEYSIGHT TECH,2016-01-31,EPS,QTR,2016-02-18,0.550
...,...,...,...,...,...,...,...
319468,TJX,TJX,2022-04-30,SAL,QTR,2022-05-18,11406.474
319469,TJX,TJX,2022-07-31,SAL,QTR,2022-08-17,11843.008
319470,TJX,TJX,2022-10-31,SAL,QTR,2022-11-16,12166.286
319471,TJX,TJX,2023-01-31,SAL,QTR,2023-02-22,14520.000


In [49]:
df_actual["PENDS"] = pd.to_datetime(df_actual["PENDS"])


df_actual.loc[:, "QUARTER"] = df_actual["PENDS"].dt.quarter
df_actual.loc[:, "YEAR"] = df_actual["PENDS"].dt.year

# For df_actual
df_actual["QUARTER_YEAR"] = df_actual["PENDS"].dt.to_period("Q").astype(str)


In [50]:
condition = (df_actual["OFTIC"] == "AES") & (df_actual["MEASURE"] == "ROE")
df_actual.loc[condition]

,OFTIC,CNAME,PENDS,MEASURE,PDICITY,ANNDATS,VALUE,QUARTER,YEAR,QUARTER_YEAR


In [51]:
df_actual[(df_actual["OFTIC"] == "PAYC") & (df_actual["MEASURE"] == "ROE")]

,OFTIC,CNAME,PENDS,MEASURE,PDICITY,ANNDATS,VALUE,QUARTER,YEAR,QUARTER_YEAR


In [52]:
appl = df_actual[df_actual["OFTIC"] == "AAPL"]
appl.isna().sum()

OFTIC           0
CNAME           0
PENDS           0
MEASURE         0
PDICITY         0
ANNDATS         0
VALUE           0
QUARTER         0
YEAR            0
QUARTER_YEAR    0
dtype: int64

In [53]:
no_missing_comp = []
for ticker in comp_list:
    if df_actual[df_actual["OFTIC"] == ticker].isna().sum().sum() == 0:
        no_missing_comp.append(ticker)


In [54]:
no_missing_comp

['PAYC',
 'ANET',
 'MTCH',
 'AAPL',
 'AAP',
 'ABBV',
 'ABT',
 'TAP',
 'CB',
 'ACN',
 'ADM',
 'AEP',
 'AES',
 'AFL',
 'SWKS',
 'HES',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'HON',
 'ALLE',
 'ALGN',
 'ALK',
 'ALL',
 'PLD',
 'AMD',
 'AMGN',
 'AMP',
 'AAL',
 'AMT',
 'AMZN',
 'ANSS',
 'AVGO',
 'APA',
 'APD',
 'APH',
 'ARE',
 'ABC',
 'ADP',
 'AVB',
 'ATVI',
 'AVY',
 'AWK',
 'AXP',
 'PNW',
 'BA',
 'BAX',
 'BR',
 'RHI',
 'BDX',
 'VZ',
 'BIO',
 'BK',
 'BRK.B',
 'WRB',
 'BLK',
 'BMY',
 'BSX',
 'BWA',
 'BXP',
 'CAT',
 'CBOE',
 'C',
 'CHTR',
 'CAH',
 'CDW',
 'CE',
 'FIS',
 'CF',
 'CFG',
 'JPM',
 'CHRW',
 'CVX',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMS',
 'CNC',
 'COF',
 'CPRT',
 'CPT',
 'CHD',
 'CRL',
 'CSX',
 'CTLT',
 'CTSH',
 'CMI',
 'D',
 'DAL',
 'DD',
 'DFS',
 'DGX',
 'DIS',
 'DISH',
 'DLR',
 'DHR',
 'DPZ',
 'DG',
 'DOV',
 'DOW',
 'OMC',
 'DHI',
 'DTE',
 'DUK',
 'DVN',
 'MS',
 'DXCM',
 'EBAY',
 'CDNS',
 'ECL',
 'ED',
 'EFX',
 'ATO',
 'PKI',
 'EL',
 'EMN',
 'EMR

In [55]:
measures = df_actual["MEASURE"].unique()
companies = df_actual["OFTIC"].unique()
companies_to_drop = []

for company in companies:
    for measure in measures:
        try:
            condition = (df_actual["OFTIC"] == company) & (df_actual["MEASURE"] == measure)
            subset = df_actual.loc[condition, "VALUE"]
            mask = ~subset.isnull() # invert the mask to get the non-null values
        except Exception as e:
            print(e)
            
        if mask.any():
            try:
                # Perform interpolation
                interpolation_function = interpolate.interp1d(subset[mask].index, subset[mask].values, fill_value='extrapolate')
                df_actual.loc[condition, "VALUE"] = interpolation_function(subset.index)
                print(f"Interpolated values in {company}, {measure}.")
            except Exception as e:
                print(e)
                companies_to_drop.append(company)
                #df_actual.loc[condition, "VALUE"] = default_value
                print(f"No non-null values in {company}, {measure}.")
        else:
            companies_to_drop.append(company)
            #df_actual.loc[condition, "VALUE"] = default_value
            print(f"No non-null values in {company}, {measure}.")

Interpolated values in KEYS, EPS.
Interpolated values in KEYS, BPS.
Interpolated values in KEYS, DPS.
Interpolated values in KEYS, EBI.
Interpolated values in KEYS, EBT.
Interpolated values in KEYS, GPS.
Interpolated values in KEYS, NAV.
Interpolated values in KEYS, NET.
Interpolated values in KEYS, ROE.
Interpolated values in KEYS, SAL.
Interpolated values in AAPL, EPS.
Interpolated values in AAPL, BPS.
Interpolated values in AAPL, DPS.
Interpolated values in AAPL, EBI.
Interpolated values in AAPL, EBT.
Interpolated values in AAPL, GPS.
Interpolated values in AAPL, NAV.
Interpolated values in AAPL, NET.
Interpolated values in AAPL, ROE.
Interpolated values in AAPL, SAL.
Interpolated values in ADSK, EPS.
Interpolated values in ADSK, BPS.
Interpolated values in ADSK, DPS.
Interpolated values in ADSK, EBI.
Interpolated values in ADSK, EBT.
Interpolated values in ADSK, GPS.
Interpolated values in ADSK, NAV.
Interpolated values in ADSK, NET.
Interpolated values in ADSK, ROE.
Interpolated v

In [56]:
print(len(companies_to_drop))
#df_actual = df_actual[~df_actual['OFTIC'].isin(companies_to_drop)]
print(companies_to_drop)

0
[]


In [57]:
df_actual.isnull().sum()

OFTIC           0
CNAME           0
PENDS           0
MEASURE         0
PDICITY         0
ANNDATS         0
VALUE           0
QUARTER         0
YEAR            0
QUARTER_YEAR    0
dtype: int64

In [58]:
# Drawing value-pends graph for one company and all measure

def draw_measure_pends(company, df):
    unique_measures = df["MEASURE"].unique()

    num_measures = len(unique_measures)
    num_cols = 3  
    num_rows = (num_measures + num_cols - 1) // num_cols  

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 25))

    for i, measure in enumerate(unique_measures):
        row, col = i // num_cols, i % num_cols
        x = df[(df['OFTIC'] == company) & (df['MEASURE'] == measure)]

        if num_rows == 1:
            axes[col].plot(x['PENDS'], x['VALUE'])
            axes[col].set_xlabel('Quarter')
            axes[col].set_ylabel('Ratio')
            axes[col].set_title(f'{measure} - Quarter Graph')
        else:
            axes[row, col].plot(x['PENDS'], x['VALUE'])
            axes[row, col].set_xlabel('Quarter')
            axes[row, col].set_ylabel('Ratio')
            axes[row, col].set_title(f'{measure} - Quarter Graph')

    for i in range(num_measures, num_rows * num_cols):
        fig.delaxes(axes.flatten()[i])

    plt.tight_layout()

    plt.show()

In [59]:
draw_measure_pends("GOOGL",df_actual)

KeyboardInterrupt: 

In [ ]:
df_actual

,OFTIC,CNAME,PENDS,MEASURE,PDICITY,ANNDATS,VALUE,QUARTER,YEAR,QUARTER_YEAR
66,KEYS,KEYSIGHT TECH,2015-01-31,EPS,QTR,2015-02-19,0.560,1,2015,2015Q1
67,KEYS,KEYSIGHT TECH,2015-04-30,EPS,QTR,2015-05-19,0.700,2,2015,2015Q2
68,KEYS,KEYSIGHT TECH,2015-07-31,EPS,QTR,2015-08-19,0.550,3,2015,2015Q3
69,KEYS,KEYSIGHT TECH,2015-10-31,EPS,QTR,2015-11-19,0.710,4,2015,2015Q4
70,KEYS,KEYSIGHT TECH,2016-01-31,EPS,QTR,2016-02-18,0.550,1,2016,2016Q1
...,...,...,...,...,...,...,...,...,...,...
319468,TJX,TJX,2022-04-30,SAL,QTR,2022-05-18,11406.474,2,2022,2022Q2
319469,TJX,TJX,2022-07-31,SAL,QTR,2022-08-17,11843.008,3,2022,2022Q3
319470,TJX,TJX,2022-10-31,SAL,QTR,2022-11-16,12166.286,4,2022,2022Q4
319471,TJX,TJX,2023-01-31,SAL,QTR,2023-02-22,14520.000,1,2023,2023Q1


In [ ]:
df_actual.to_csv("interpolated_actuals.csv", index=False)